In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import spacy
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

from tqdm._tqdm_notebook import tqdm, tqdm_notebook, tnrange
from S3_read_write import load_df_s3, save_df_s3

from IPython.display import Image
from IPython.core.display import HTML 

In [3]:
tqdm_notebook.pandas('Progress')

In [4]:
bucket_name = 'amazon-reviews-project'

# Load Amazon Reviews Data

In [30]:
reviews = load_df_s3(bucket_name, 'amazon_reviews/reviews_data_clean', filetype='text', sep='|')

In [31]:
reviews.shape    # 585,444 records

(585444, 8)

In [32]:
reviews.head()

,asin,helpful,reviewText,overall,summary,description,title,categories_clean
0,0929619730,"[0, 0]",B-flax-D is a re...,5.0,Dpes the job well,Contains Organic...,New Generation B...,Health & Persona...
1,0978559088,"[1, 1]",Studies show tha...,4.0,"Fast shipping, g...",Everyone knows t...,Nutrihill Resver...,Health & Persona...
2,0978559088,"[1, 1]",I started taking...,5.0,Bioavailability ...,Everyone knows t...,Nutrihill Resver...,Health & Persona...
3,0978559088,"[0, 1]",I tried Nutrihil...,1.0,Other Resveratro...,Everyone knows t...,Nutrihill Resver...,Health & Persona...
4,0978559088,"[0, 0]",I really liked t...,5.0,I can't find thi...,Everyone knows t...,Nutrihill Resver...,Health & Persona...


In [33]:
reviews.dtypes

asin                 object
helpful              object
reviewText           object
overall             float64
summary              object
description          object
title                object
categories_clean     object
dtype: object

## Data Cleaning

In [52]:
reviews.categories_clean.unique()[:10]

array(['Health & Personal Care, Vitamins & Dietary Supplements, Multi & Prenatal Vitamins, Multiple Vitamin-Mineral Supplements',
       'Health & Personal Care, Vitamins & Dietary Supplements, Supplements, Antioxidants, Resveratrol',
       'Health & Personal Care, Vitamins & Dietary Supplements, Multi & Prenatal Vitamins, Multivitamins',
       'Health & Personal Care, Vitamins & Dietary Supplements, Vitamins, Vitamin B, B3 (Niacin)',
       'Health & Personal Care, Vitamins & Dietary Supplements, Herbal Supplements',
       'Health & Personal Care, Vitamins & Dietary Supplements, Herbal Supplements, Green Tea',
       'Health & Personal Care, Vitamins & Dietary Supplements, Weight Loss, Supplements, Green Coffee Bean Extract',
       'Health & Personal Care, Vitamins & Dietary Supplements, Weight Loss, Supplements',
       'Health & Personal Care, Vitamins & Dietary Supplements, Supplements, Antioxidants, CoQ10',
       'Health & Personal Care, Vitamins & Dietary Supplements, Herbal

The catergories' list indicates that there may be some reviews in the dataset unrelated to health supplements.  Let's get rid of these.

In [35]:
reviews[reviews.categories_clean.str.contains('CDs & Vinyl')].title.unique()

array(['Liturgy of St. John Chrysostom', 'Origins',
       'Sounds of the Earth: Soft Ocean Sounds', 'Bali',
       'Tranquil Waters', 'Bach: St. John Passion, BWV 245',
       '21st Century Soul', 'Bodies for Strontium', "John's Bunch",
       'An Evening of Paganini', "John's Other Bunch",
       'Sus Mas Grandes Exitos', 'Complex Simplicity',
       'Kidnapped By Neptune', 'Roman Chant / Easter Vespers', 'Dead 60s',
       "Cilla in the 60's", 'Chromium', 'Letters From the Vitamin Sea',
       'The Stinging Nettles', 'Tendres Annees 60', 'Wehiwehi Hawaii',
       'none'], dtype=object)

In [36]:
len(reviews[reviews.categories_clean.str.contains('CDs & Vinyl')])

263

The product titles shown above are all music albums/songs.

In [37]:
reviews_filt = reviews[~(reviews.categories_clean.str.contains('CDs & Vinyl'))]   # remove rows with category including 'CDs & Vinyl'

In [51]:
reviews_filt.categories_clean.unique()[:10]

array(['Health & Personal Care, Vitamins & Dietary Supplements, Multi & Prenatal Vitamins, Multiple Vitamin-Mineral Supplements',
       'Health & Personal Care, Vitamins & Dietary Supplements, Supplements, Antioxidants, Resveratrol',
       'Health & Personal Care, Vitamins & Dietary Supplements, Multi & Prenatal Vitamins, Multivitamins',
       'Health & Personal Care, Vitamins & Dietary Supplements, Vitamins, Vitamin B, B3 (Niacin)',
       'Health & Personal Care, Vitamins & Dietary Supplements, Herbal Supplements',
       'Health & Personal Care, Vitamins & Dietary Supplements, Herbal Supplements, Green Tea',
       'Health & Personal Care, Vitamins & Dietary Supplements, Weight Loss, Supplements, Green Coffee Bean Extract',
       'Health & Personal Care, Vitamins & Dietary Supplements, Weight Loss, Supplements',
       'Health & Personal Care, Vitamins & Dietary Supplements, Supplements, Antioxidants, CoQ10',
       'Health & Personal Care, Vitamins & Dietary Supplements, Herbal

In [39]:
reviews_filt[reviews_filt.categories_clean.str.contains('Software')]

,asin,helpful,reviewText,overall,summary,description,title,categories_clean
3639,B00009QP4Q,"[2, 2]",The company has ...,5.0,lives up to its ...,Alpha Five's QLi...,none,Health & Persona...
50015,B0002TIEQQ,"[0, 0]",I ordered this f...,1.0,waste of money,Self help tutori...,none,Health & Persona...


In [40]:
reviews_filt = reviews_filt[~(reviews_filt.categories_clean.str.contains('Software'))]

In [41]:
len(reviews_filt)

585179

In [53]:
# Get rid of reviews of pet-related products
search_for = [' pet ', ' cat ', ' dog ']
pattern = '|'.join(search_for)
reviews_filt.title.str.contains(pattern, case=False).sum()

277

In [50]:
reviews_filt[reviews_filt.title.str.contains(pattern, case=False)]['title'].values[:10]

array(['Power - Mune Tuna Flavor Pet Herbal Supplement From Vetvittles.com',
       'Power - Mune Tuna Flavor Pet Herbal Supplement From Vetvittles.com',
       'Power - Mune Tuna Flavor Pet Herbal Supplement From Vetvittles.com',
       'AniMed Witch Hazel 86-Percent Multi-Species Pet Supplement',
       'AniMed Witch Hazel 86-Percent Multi-Species Pet Supplement',
       'AniMed Witch Hazel 86-Percent Multi-Species Pet Supplement',
       'AniMed Witch Hazel 86-Percent Multi-Species Pet Supplement',
       'AniMed Witch Hazel 86-Percent Multi-Species Pet Supplement',
       'AniMed Witch Hazel 86-Percent Multi-Species Pet Supplement',
       'Composure Liquid for Dogs and Cat (188 SERVINGS)'], dtype=object)

In [54]:
# Get rid of all pet products
reviews_filt = reviews_filt[~(reviews_filt.title.str.contains(pattern, case=False))]

In [55]:
# saving the cleaned dataframe
save_df_s3(df=reviews_filt, bucket_name=bucket_name, filepath='amazon_reviews/reviews_data_clean_v2.feather')

In [56]:
reviews_filt.asin.nunique()     # 48,535 unique products and 585,179 reviews

48501

## Examine One Observation

In [57]:
example = reviews_filt.iloc[0]

In [58]:
example.asin     # Amazon Standard Identification Number

'0929619730'

In [59]:
example.title     # this is the product's name

'New Generation B-Flax-D'

In [60]:
example.categories_clean   # previously filtered/curated categories of interest

'Health & Personal Care, Vitamins & Dietary Supplements, Multi & Prenatal Vitamins, Multiple Vitamin-Mineral Supplements'

In [61]:
example.description       # product description provided by the seller

'Contains Organic Cold-Milled Flaxseed\nValuable source of soluble and insoluble fiber\nProvides Omega-3 essential fats, and many other nutrients to help achieve and maintain optimal bowel function.\n\nContains Vitamin B12\nB12 helps prevent nerve damage\nB12 aids in healthy cell formation.\nB12 helps prevent anemia\n\nContains Vitamin D\nVitamin D assists the body in the absorption of important minerals like calcium.\n\nContains Seleno-yeast\nA source of selenium, a mineral with powerful anti-viral and disease-fighting properties.\n\nContains Vitamin K2\nMenaQ7TM provides vitamin K2 (menaquinone), extracted and concentrated from natto without solvents. Vitamin K2 prevents arterial calcification and promotes strong bones by improving cross-linking of osteocalcin, a protein found in bones. The amount here has been clinically shown not to interfere with blood anti-coagulant medication. \n\nServing Size:\n1/4 Cup (30 Grams)\n\nServings Per Container:\n30 Servings per container\n\nNet Wt. 

In [62]:
example.summary      # review title

'Dpes the job well'

In [63]:
example.reviewText   # review content

'B-flax-D is a regular at our house. It does its job simply and with good results. It is reasonable, lasts a long time, and is able to be obtained with free shipping if you hunt around. Good product, good price, good results.'

Here's what the actual review looks like:

In [64]:
example.overall     # the rating provided by the reviewer

5.0

In [29]:
example.helpful

'[0, 0]'

In [27]:
Image(url= "images/amazon_review_screenshot.png")

# Data Pre-processing

Let us start off using only the title (`summary`) and body (`reviewText`) of each review.

In [36]:
%%time
df = load_df_s3(bucket_name, filepath='amazon_reviews/reviews_data_clean_v2.feather', filetype='feather')

CPU times: user 2.98 s, sys: 4.02 s, total: 7 s
Wall time: 19 s


In [37]:
df.dtypes

asin                 object
helpful              object
reviewText           object
overall             float64
summary              object
description          object
title                object
categories_clean     object
dtype: object

In [38]:
df.drop(['helpful', 'overall', 'title', 'categories_clean', 'description'], axis=1, inplace=True)

In [39]:
df.head()

,asin,reviewText,summary
0,0929619730,B-flax-D is a regular at our house. It does it...,Dpes the job well
1,0978559088,Studies show that Resveratrol is poorly absorb...,"Fast shipping, good communication"
2,0978559088,I started taking this after both my parents di...,Bioavailability is the key
3,0978559088,"I tried Nutrihill, but did not feel any of the...",Other Resveratrol Supplements are Better
4,0978559088,I really liked this product because it stayed ...,"I can't find this product any longer, and I wi..."


In [40]:
# for each review, concatenate the review title and body
df.reviewText = df.summary + '. ' + df.reviewText

In [41]:
pd.set_option('max_colwidth', 200)
df.head()

,asin,reviewText,summary
0,0929619730,"Dpes the job well. B-flax-D is a regular at our house. It does its job simply and with good results. It is reasonable, lasts a long time, and is able to be obtained with free shipping if you hunt ...",Dpes the job well
1,0978559088,"Fast shipping, good communication. Studies show that Resveratrol is poorly absorbed when taken by pill, but lozenges are very effectively absorbed. Hardly any companies are selling lozenges. This ...","Fast shipping, good communication"
2,0978559088,Bioavailability is the key. I started taking this after both my parents died of cancer as it supposed to enhance your immune system - the story on 60 Minutes on resveratrol was incredibly inspirin...,Bioavailability is the key
3,0978559088,"Other Resveratrol Supplements are Better. I tried Nutrihill, but did not feel any of the supposed health benefits. I started reading and realized that even though buccal delivery is the best, the ...",Other Resveratrol Supplements are Better
4,0978559088,"I can't find this product any longer, and I wish I could.. I really liked this product because it stayed in my mouth for a long time and I felt it was probably doing some good. I take a number of...","I can't find this product any longer, and I wish I could."


Let's drop the `summary` column now:

In [42]:
df.drop(['summary'], axis=1, inplace=True)

In [43]:
df.head()

,asin,reviewText
0,0929619730,"Dpes the job well. B-flax-D is a regular at our house. It does its job simply and with good results. It is reasonable, lasts a long time, and is able to be obtained with free shipping if you hunt ..."
1,0978559088,"Fast shipping, good communication. Studies show that Resveratrol is poorly absorbed when taken by pill, but lozenges are very effectively absorbed. Hardly any companies are selling lozenges. This ..."
2,0978559088,Bioavailability is the key. I started taking this after both my parents died of cancer as it supposed to enhance your immune system - the story on 60 Minutes on resveratrol was incredibly inspirin...
3,0978559088,"Other Resveratrol Supplements are Better. I tried Nutrihill, but did not feel any of the supposed health benefits. I started reading and realized that even though buccal delivery is the best, the ..."
4,0978559088,"I can't find this product any longer, and I wish I could.. I really liked this product because it stayed in my mouth for a long time and I felt it was probably doing some good. I take a number of..."


In [44]:
pd.set_option('max_colwidth', 20)

## Remove Missing Reviews

In [45]:
df.reviewText.isnull().sum()    # 73 reviews have neither a review body text, nor a review title

73

In [46]:
# drop reviews with no text
df = df[~(df.reviewText.isnull())]

In [47]:
df.asin.isnull().sum()

0

Let's look at a few actual review texts:

In [48]:
df.reviewText.iloc[np.random.randint(0, len(df))]

'it works!. started taking this on recommendation from a friend. and it works! it helped a lot in arresting the hair fall. But be aware that vitamin/mineral deficiency is only one among many a reasons of hair fall (my doctor told me so) so what is good for me may not be good for all.'

In [49]:
df.reviewText.iloc[np.random.randint(0, len(df))]

"Good value, good quality. I take Vitamin C daily so a good quality supplement is very important to me.I was happy to see the Viva Labs Vitamin C offered on the vine program.I was especially happy to see a non-gmo supplement sourced from a non-asian source.Finding a good vitamin C supplement at a reasonable price that contains the bioflavonoids needed for your body to properly use and assimilate the vitamin C isn't easy.  This product just ended that search.The capsules are large but I had no problem swallowing them with water (and I have a terrible gag reflex).  They didn't get that gummy texture when the water hits them like some capsules.  That helped get them down quickly.I take vitamin C with food to avoid stomach problems and didn't have any problem with these either.I took off 1 star for packaging quality issue and less than optimal customer service.  The word &#34;bioflavonoids&#34; is misspelled on the bottle.  That led me to worry about whether the product was authentic and/o

In [50]:
df.reviewText.iloc[np.random.randint(0, len(df))]

"I really notice a difference. in my level of energy for working out (don't get fatigued as quickly) and I have noticed that I am seeing more muscle tone"

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 584829 entries, 0 to 584901
Data columns (total 2 columns):
asin          584829 non-null object
reviewText    584829 non-null object
dtypes: object(2)
memory usage: 13.4+ MB


## Pre-processing for LDA

In [52]:
text = list(df.reviewText.values)    # make an iterable to store only the review text

In [53]:
len(text)

584829

In [54]:
# look at a few sample reviews
for rev in text[:4]:
    print(rev, '\n')

Dpes the job well. B-flax-D is a regular at our house. It does its job simply and with good results. It is reasonable, lasts a long time, and is able to be obtained with free shipping if you hunt around. Good product, good price, good results. 

Fast shipping, good communication. Studies show that Resveratrol is poorly absorbed when taken by pill, but lozenges are very effectively absorbed. Hardly any companies are selling lozenges. This company promises 99% purity and has fast shipping and good communication. I can't comment on the quality of product because I'm not a chemist but they seem to be legitimate. 

Bioavailability is the key. I started taking this after both my parents died of cancer as it supposed to enhance your immune system - the story on 60 Minutes on resveratrol was incredibly inspiring. Doing some research on the Internet, it is indicated that taking resveratrol in lozenge form is preferable as it is broken down by stomach acids.  The ez-melt formula recommended in a

In [55]:
nlp = spacy.load('en')

The helper functions below are from:

http://nbviewer.jupyter.org/github/skipgram/modern-nlp-in-python/blob/master/executable/Modern_NLP_in_Python.ipynb

### Phrase Detection

In order to use `gensim`'s `Phrases` class to detect natural combinations of words (like 'vanilla ice cream'), we need to format our text into a list of sentences, with each sentence being a list of words.  This process takes a large amount of processing time (for reference, the times shown under the cells are for running the tasks on a c5.9xlarge EC2 instance), so `text` has been split into 3 parts.

#### Unigram Sentences

In [56]:
# split text into 3 parts
text_first = text[:194943]
text_second = text[194943:389886]
text_third = text[389886:]

In [27]:
unigram_sentences = []
for parsed_review in tqdm(nlp.pipe(text_first, batch_size=20000, n_threads=36)):
    for sent in parsed_review.sents:
        unigram_sentences.append([token.lemma_ for token in sent if not (token.is_space or token.is_punct)])

194943it [30:39, 105.95it/s]


In [29]:
len(unigram_sentences)

1206074

In [30]:
# Save progress...
unigram_sentences_savedf = pd.DataFrame([u' '.join(sentence) for sentence in unigram_sentences], columns=['unigram_sentences'])
save_df_s3(unigram_sentences_savedf, bucket_name, 'amazon_reviews/unigram_sentences.feather')

In [30]:
# [OPTIONAL - only do this if the process got interrupted] 
# load saved unigram_sentences_savedf
unigram_sentences_savedf = load_df_s3(bucket_name, filepath='amazon_reviews/unigram_sentences.feather', filetype='feather')
unigram_sentences = unigram_sentences_savedf.unigram_sentences.tolist()
unigram_sentences = [sentence.split(' ') for sentence in unigram_sentences]

In [57]:
# Continue appending to unigram_sentences: 
for parsed_review in tqdm(nlp.pipe(text_second, batch_size=10000, n_threads=36)):
    for sent in parsed_review.sents:
        unigram_sentences.append([token.lemma_ for token in sent if not (token.is_space or token.is_punct)])

194943it [29:51, 108.84it/s]


In [58]:
len(unigram_sentences)

2385921

In [59]:
# Save progress...
del unigram_sentences_savedf
unigram_sentences_savedf = pd.DataFrame([u' '.join(sentence) for sentence in unigram_sentences], columns=['unigram_sentences'])
save_df_s3(unigram_sentences_savedf, bucket_name, 'amazon_reviews/unigram_sentences.feather')

In [63]:
# Continue appending to unigram_sentences: 
for parsed_review in tqdm(nlp.pipe(text_third, batch_size=10000, n_threads=36)):
    for sent in parsed_review.sents:
        unigram_sentences.append([token.lemma_ for token in sent if not (token.is_space or token.is_punct)])

194943it [29:51, 108.80it/s]


In [64]:
len(unigram_sentences)     # sentences in the review corpus

3605491

In [65]:
# Save progress...
del unigram_sentences_savedf
unigram_sentences_savedf = pd.DataFrame([u' '.join(sentence) for sentence in unigram_sentences], columns=['unigram_sentences'])
save_df_s3(unigram_sentences_savedf, bucket_name, 'amazon_reviews/unigram_sentences.feather')

In [ ]:
# to be continued from here...

In [208]:
# Train a first-order phrase detector
bigram_model = Phrases(unigram_sentences, min_count=5)

# Transform unigram sentences into bigram sentences
# Paired words are connected by an underscore, e.g. ice_cream
bigram_sentences = []
for sentence in unigram_sentences:
    bigram_sentences.append(bigram_model[sentence])

In [267]:
# Train a second-order phrase detector
trigram_model = Phrases(bigram_sentences, min_count=5)

# Transform bigram sentences into trigram sentences
trigram_sentences = []
for sentence in bigram_sentences:
    trigram_sentences.append(trigram_model[sentence])

# remove any remaining stopwords
trigram_sentences = [[word for word in sentence if word not in nlp.Defaults.stop_words] for sentence in trigram_sentences]

/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [292]:
# the trigrams will be saved in a dataframe with a single column.
# each row is one sentence from any review
# each sentence is a single string separated by a single space.
trigram_sentences_savedf = pd.DataFrame([u' '.join(sentence) for sentence in trigram_sentences], columns=['preprocessed_review'])
save_df_s3(trigram_sentences_savedf, bucket_name, 'amazon_reviews/preprocessed_reviews.feather')

In [293]:
trigram_sentences_savedf.head()

,preprocessed_review
0,dpe the job well
1,b flax d be a re...
2,-PRON- do -PRON-...
3,-PRON- be reason...
4,good product goo...


In [209]:
unigram_sentences[0:4]

[['dpe', 'the', 'job', 'well'],
 ['b', 'flax', 'd', 'be', 'a', 'regular', 'at', '-PRON-', 'house'],
 ['-PRON-', 'do', '-PRON-', 'job', 'simply', 'and', 'with', 'good', 'result'],
 ['-PRON-',
  'be',
  'reasonable',
  'last',
  'a',
  'long',
  'time',
  'and',
  'be',
  'able',
  'to',
  'be',
  'obtain',
  'with',
  'free',
  'shipping',
  'if',
  '-PRON-',
  'hunt',
  'around']]

In [210]:
bigram_sentences[0:4]

[['dpe', 'the', 'job', 'well'],
 ['b', 'flax', 'd', 'be', 'a', 'regular', 'at', '-PRON-', 'house'],
 ['-PRON-', 'do', '-PRON-', 'job', 'simply', 'and', 'with', 'good', 'result'],
 ['-PRON-',
  'be',
  'reasonable',
  'last',
  'a',
  'long_time',
  'and',
  'be',
  'able_to',
  'be',
  'obtain',
  'with',
  'free_shipping',
  'if',
  '-PRON-',
  'hunt',
  'around']]

In [211]:
example = [word for sentence in unigram_sentences for word in sentence]  # flattened list

In [212]:
len(example)

373578

In [213]:
example[:10]

['dpe', 'the', 'job', 'well', 'b', 'flax', 'd', 'be', 'a', 'regular']

In [214]:
sample_bigr = bigram_model[example]

/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [215]:
len(sample_bigr)

354716

In [216]:
sample_bigr[:10]

['dpe', 'the', 'job', 'well', 'b', 'flax', 'd', 'be', 'a', 'regular']

In [217]:
sample_trigr = trigram_model[sample_bigr]

/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [218]:
len(sample_trigr)

349647

In [219]:
sample_trigr[:10]

['dpe', 'the', 'job', 'well', 'b', 'flax', 'd', 'be', 'a', 'regular']

In [241]:
bigr_set = set([word for word in sample_bigr if '_' in word])   # bi-grams

# print arbitrary 20 bigrams
count = 0
for e in bigr_set:
    if count <= 20:
        print(e)
        count += 1
    else:
        break

cell_phone
allergic_reaction
prostate_health
in_conjunction
what_happen
knee_pain
fat_loss
very_disappointed
point_where
whole_food
bald_spot
zone_perfect
sugar_free
brown_spot
pycnogenol_gel
fine_line
fda_approve
next_day
such_as
belt_clip
extra_energy


In [224]:
trigr_set = set([word for word in sample_trigr if '_' in word])   # tri-grams

In [247]:
# print 20 trigrams containing 'no' or 'not'
for e in (trigr_set - bigr_set):
    if ('no_' in e or 'not_' in e):
        print(e)

not_go_wrong
not_live_without
no_longer_carry
no_matter_what
have_no_idea
do_not_know_why
do_not_know_how
no_side_affect
no_jet_lag
do_not_notice_any
no_issue
no_side_effect
no_improvement
do_not_bother
not_sure


The list below shows the additional "trigrams" captured by the model.  Note that all of these are not exactly trigrams - there are some bigrams here as well.  This is because in the formula for calculating the threshold for forming a phrase, the total vocabulary of the text corpus is a factor.  We trained the trigrams model on a different corpus than the one we used for the bigrams model.  Because of this, some additional two-word phrases also get detected.

Some potentially useful examples of phrases detected here:

* no_jet_lag
* not_go_wrong
* no_complaint
* no_side_effect

The meanings of the above words change if we don't treat them as part of a single phrase.

In [ ]:
for parsed_review in tqdm_notebook(nlp.pipe(text, batch_size=10000, n_threads=36)):
    unigram_review = [token.lemma_ for token in parsed_review if not (token.is_space or token.is_punct)]

In [ ]:
%%time
unigram_sentences_all = [sentence for sentence in lemmatized_sentence_corpus(text)]

In [ ]:
%%time

for parsed_review in nlp.pipe(text, batch_size=1000, n_threads=4):

    # lemmatize the text, removing punctuation and whitespace
    unigram_review = [token.lemma_ for token in parsed_review if not (token.is_space or token.is_punct)]

    # apply the first-order and second-order phrase models
    bigram_review = bigram_model[unigram_review]
    trigram_review = trigram_model[bigram_review]

    # remove any remaining stopwords
    trigram_review = [term for term in trigram_review if term not in spacy.en.STOPWORDS]

    # write the transformed review as a line in the new file
    trigram_review = u' '.join(trigram_review)
    f.write(trigram_review + '\n')

In [ ]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 0 == 1:

    with codecs.open(trigram_reviews_filepath, 'w', encoding='utf_8') as f:
        
        for parsed_review in nlp.pipe(line_review(review_txt_filepath),
                                      batch_size=10000, n_threads=4):
            
            # lemmatize the text, removing punctuation and whitespace
            unigram_review = [token.lemma_ for token in parsed_review
                              if not punct_space(token)]
            
            # apply the first-order and second-order phrase models
            bigram_review = bigram_model[unigram_review]
            trigram_review = trigram_model[bigram_review]
            
            # remove any remaining stopwords
            trigram_review = [term for term in trigram_review
                              if term not in spacy.en.STOPWORDS]
            
            # write the transformed review as a line in the new file
            trigram_review = u' '.join(trigram_review)
            f.write(trigram_review + '\n')